In [ ]:

from google.colab import drive

In [ ]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,BatchNormalization,Dropout
import numpy as np
import pandas as pd
import cv2
import os

In [ ]:

drive.mount('/content/drive')#connect drive to colab

In [ ]:
Datadir='/content/drive/MyDrive/Mini_Project_V'
category=["PNEUMONIA","NORMAL"]
dataset=[]
size=150
ct=0
ct1=0
for types in category:
    path=os.path.join(Datadir,types)
    ex=0
    for img in os.listdir(path):
        if types=='PNEUMONIA':
            ex=1
            ct=ct+1;
        else :
            ct1=ct1+1;
            ex=0
        try :
            img_arr=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
            new_img=cv2.resize(img_arr ,(size,size),interpolation=cv2.INTER_AREA)
            dataset.append([new_img,ex])
        except Exception as e:
            pass

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
plt.imshow(dataset[0][0])

In [ ]:

import random
random.shuffle(dataset)
X=[]
Y=[]
for features,label in dataset:
  X.append(features)
  Y.append(label)

X=np.array(X).reshape(-1,size,size,1)


X=X/225
Y=np.array(Y)

In [ ]:
from sklearn.model_selection import train_test_split 
X_train,X_test,Y_train,Y_test=train_test_split(X, Y, test_size=0.2, random_state=50)

In [ ]:

model=Sequential()
model.add(Flatten(input_shape=(size,size)))
model.add(BatchNormalization(axis=1))
model.add(Dense(408,activation='relu'))
model.add(BatchNormalization(axis=1))
model.add(Dense(104,activation='relu'))
model.add(BatchNormalization(axis=1))plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
model.add(Dense(32,activation='relu'))
model.add(Dense(2,activation='sigmoid'))

In [ ]:

model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])#0.01

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,CSVLogger
filepath="/content/drive/MyDrive/saved_models/CatvsDog-{epoch:02d}-{val_accuracy:.2f}.hdf5"

# save the model with the best validation (development) accuracy till now
earlystopping = EarlyStopping(monitor='val_loss',patience=8,verbose=1)


callback_list=[earlystopping]

In [ ]:

history=model.fit(X_train,Y_train,epochs=15,validation_split=0.2,callbacks=callback_list)

In [ ]:
y_prob=model.predict(X_test)
y_pred=y_prob.argmax(axis=1)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,y_pred)

In [ ]:

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])


In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

In [ ]:
#saving model
model.save_weights("/content/drive/MyDrive/Mini_Project_V/TESTANN.hdf5",overwrite=True)
     

In [ ]:
from sklearn.metrics import confusion_matrix 
import seaborn as sns

sns.heatmap(confusion_matrix(Y_test, y_pred), annot=True, cmap='Blues')

In [ ]:
model1=Sequential()

model1.add(Conv2D(16,(3,3),activation='relu',input_shape=(size,size,1)))
model1.add(MaxPool2D((2,2)))
model1.add(Conv2D(32,(3,3),activation='relu'))
model1.add(MaxPool2D((2,2)))
model1.add(Conv2D(64,(3,3),activation='relu'))
model1.add(MaxPool2D((2,2)))
model1.add(Flatten())
model1.add(Dense(16,activation='relu'))
model1.add(BatchNormalization(axis=1))
model1.add(Dense(1,activation='sigmoid'))

In [ ]:
Y_train=np.reshape(Y_train,(3791,1))
Y_train.shape

In [ ]:
model1.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
history2=model1.fit(X_train,Y_train,epochs=15,batch_size=64,validation_split=0.2)

In [ ]:
y_prob=model1.predict(X_test)
y_prob=y_prob>0.5
accuracy_score(Y_test,y_pred)

In [ ]:

plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])

In [ ]:
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])

In [ ]:
sns.heatmap(confusion_matrix(Y_test, y_prob), annot=True, cmap='Blues')


In [ ]:
#saving model
model1.save_weights("/content/drive/MyDrive/Mini_Project_V/TESTCNN.hdf5",overwrite=True)